Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [2]:
!pip install -q kaggle
!pip install -q kaggle-cli
!mkdir -p ~/.kaggle
!cp drive/MyDrive/kaggle.json ~/.kaggle/
!cat ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download avito-demand-prediction -f train.csv -p sample_data/

     |████████████████████████████████| 74 kB 1.7 MB/s 
     |████████████████████████████████| 4.2 MB 8.6 MB/s 
     |████████████████████████████████| 49 kB 4.8 MB/s 
     |████████████████████████████████| 150 kB 61.8 MB/s 
     |████████████████████████████████| 113 kB 62.2 MB/s 
  ERROR: Failed building wheel for lxml
    Running setup.py install for lxml ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-7auyvrdj/lxml_acf0de770b7746fba9fd7b18a068a464/setup.py'"'"'; __file__='"'"'/tmp/pip-install-7auyvrdj/lxml_acf0de770b7746fba9fd7b18a068a464/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-8ghasjq9/install-record.txt --si

In [3]:
import math
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn

from sklearn.model_selection import train_test_split

1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader

In [4]:
class AvitoDataset(torch.utils.data.Dataset):
    def __init__(self, filepath, train=True):
        df = pd.read_csv(filepath)
        df['activation_date'] = pd.to_datetime(df['activation_date'])
        df['month'] = df['activation_date'].dt.month
        df['day'] = df['activation_date'].dt.day
        df['weekday'] = df['activation_date'].dt.weekday

        y = df['deal_probability']
        X = df.drop(columns=['deal_probability'], axis=1).select_dtypes(exclude=['object', 'datetime64[ns]'])

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size = 0.25)

        X_train = X_train.fillna(0)
        X_test = X_test.fillna(0)

        if train:
          self.data = X_train
          self.labels = np.array(y_train)
        else:
          self.data = X_test
          self.labels = np.array(y_test)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data.iloc[index, :].values, self.labels[index]

In [5]:
batch_size = 1024

train_dataset = AvitoDataset('sample_data/train.csv.zip')
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                          shuffle=True, 
                                          #num_workers=2, 
                                          drop_last=True)

test_dataset = AvitoDataset('sample_data/train.csv.zip', False)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                          shuffle=True, 
                                          #num_workers=2, 
                                          drop_last=True)

3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

In [6]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return torch.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    # Пробовал добавить дополнительный слой с последующей нормализацией и дропаутом, но улучшения результата не удалось добиться
    # Поэтому вернулся к архитектуре, представленной на уроке
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp1 = nn.Dropout(0.2)
        self.fc2 = Perceptron(hidden_dim, 1, activation='sigmoid')
        self.double()
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.dp1(x)
        x = self.fc2(x)
        return x


# В соревновании на каггле использовалалсь метрика RMSE, поэтому реализовывал ее
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        eps = 1e-6
        loss = torch.sqrt(criterion(x, y) + eps)
        return loss

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for i, data in enumerate(dataloader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels.reshape(1024,-1))
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            loss, current = loss.item(), i * len(inputs)
            print(f"loss: {loss:>7f}  [{current:>8d}/{size:>8d}]")



def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            test_loss += loss_fn(outputs, labels.reshape(1024,-1)).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [8]:
net = FeedForward(6, 30)

optimizer = torch.optim.SGD(net.parameters(), lr=1e-3)
criterion = RMSELoss()

In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, net, criterion, optimizer)
    test(test_data_loader, net, criterion)
print("Done!")

Epoch 1
-------------------------------
loss: 0.481468  [       0/ 1127568]
loss: 0.463155  [  307200/ 1127568]
loss: 0.442622  [  614400/ 1127568]
loss: 0.427547  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.417899 

Epoch 2
-------------------------------
loss: 0.416945  [       0/ 1127568]
loss: 0.406108  [  307200/ 1127568]
loss: 0.397721  [  614400/ 1127568]
loss: 0.388491  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.371687 

Epoch 3
-------------------------------
loss: 0.370910  [       0/ 1127568]
loss: 0.362188  [  307200/ 1127568]
loss: 0.354867  [  614400/ 1127568]
loss: 0.346678  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.334741 

Epoch 4
-------------------------------
loss: 0.333797  [       0/ 1127568]
loss: 0.332681  [  307200/ 1127568]
loss: 0.326494  [  614400/ 1127568]
loss: 0.319778  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.309056 

Epoch 5
-------------------------------
loss: 0.312948  [       0/ 1127568]
loss: 0.310885  [  307200/ 1127568]
loss: 0.2957

Попробуем построить модели с другими оптимизаторами:

In [22]:
net = FeedForward(6, 30)

optimizer = torch.optim.Adam(net.parameters())
criterion = RMSELoss()

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, net, criterion, optimizer)
    test(test_data_loader, net, criterion)
print("Done!")

Epoch 1
-------------------------------
loss: 0.464453  [       0/ 1127568]
loss: 0.257404  [  307200/ 1127568]
loss: 0.254053  [  614400/ 1127568]
loss: 0.246998  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.253009 

Epoch 2
-------------------------------
loss: 0.251381  [       0/ 1127568]
loss: 0.244445  [  307200/ 1127568]
loss: 0.247974  [  614400/ 1127568]
loss: 0.240892  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.251903 

Epoch 3
-------------------------------
loss: 0.250829  [       0/ 1127568]
loss: 0.249453  [  307200/ 1127568]
loss: 0.255675  [  614400/ 1127568]
loss: 0.258463  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.252016 

Epoch 4
-------------------------------
loss: 0.245201  [       0/ 1127568]
loss: 0.249977  [  307200/ 1127568]
loss: 0.251528  [  614400/ 1127568]
loss: 0.244056  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.251075 

Epoch 5
-------------------------------
loss: 0.251464  [       0/ 1127568]
loss: 0.246134  [  307200/ 1127568]
loss: 0.2597

In [24]:
net = FeedForward(6, 30)

optimizer = torch.optim.RMSprop(net.parameters(), lr=1e-3)
criterion = RMSELoss()


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, net, criterion, optimizer)
    test(test_data_loader, net, criterion)
print("Done!")

Epoch 1
-------------------------------
loss: 0.446455  [       0/ 1127568]
loss: 0.267454  [  307200/ 1127568]
loss: 0.249551  [  614400/ 1127568]
loss: 0.236385  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.251909 

Epoch 2
-------------------------------
loss: 0.251708  [       0/ 1127568]
loss: 0.272383  [  307200/ 1127568]
loss: 0.264237  [  614400/ 1127568]
loss: 0.251765  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.252839 

Epoch 3
-------------------------------
loss: 0.244552  [       0/ 1127568]
loss: 0.243975  [  307200/ 1127568]
loss: 0.244089  [  614400/ 1127568]
loss: 0.256386  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.251878 

Epoch 4
-------------------------------
loss: 0.247568  [       0/ 1127568]
loss: 0.248801  [  307200/ 1127568]
loss: 0.252165  [  614400/ 1127568]
loss: 0.247766  [  921600/ 1127568]
Test Error: 
 Avg loss: 0.252917 

Epoch 5
-------------------------------
loss: 0.245207  [       0/ 1127568]
loss: 0.249938  [  307200/ 1127568]
loss: 0.2465

RMSprop и Adam сходятся быстрее, чем SGD. Результаты также довольно близки. Однако в сравнении с лидерами рейтинга соревнования на каггле видно, что качество модели довольно низкое. Это неудивительно с учетом небольшого количества признаков, на основе которых проводится обучение.

Аналогичное сравнение со средней ошибкой:

In [10]:
from tqdm import tqdm


#SGD
net = FeedForward(6, 30)

optimizer = torch.optim.SGD(net.parameters(), lr=1e-3)
criterion = RMSELoss()


for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels.reshape(1024, -1))
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 299:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,   300] loss: 0.480
[1,   600] loss: 0.460
[1,   900] loss: 0.441


 10%|█         | 1/10 [02:43<24:30, 163.36s/it]

[2,   300] loss: 0.412
[2,   600] loss: 0.398
[2,   900] loss: 0.386


 20%|██        | 2/10 [05:27<21:52, 164.09s/it]

...

[9,   300] loss: 0.274
[9,   600] loss: 0.274
[9,   900] loss: 0.272


 90%|█████████ | 9/10 [24:28<02:43, 163.42s/it]

[10,   300] loss: 0.271
[10,   600] loss: 0.270
[10,   900] loss: 0.269


100%|██████████| 10/10 [27:12<00:00, 163.24s/it]

Training is finished!


In [11]:
#Adam
net = FeedForward(6, 30)

optimizer = torch.optim.Adam(net.parameters())
criterion = RMSELoss()


for epoch in tqdm(range(10)):  
    running_loss = 0.0
    print()
    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels.reshape(1024, -1))
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 299:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]


[1,   300] loss: 0.322
[1,   600] loss: 0.257
[1,   900] loss: 0.255


 10%|█         | 1/10 [02:43<24:31, 163.48s/it]


[2,   300] loss: 0.254
[2,   600] loss: 0.253
[2,   900] loss: 0.252


 20%|██        | 2/10 [05:26<21:45, 163.20s/it]

...


[9,   300] loss: 0.251
[9,   600] loss: 0.250
[9,   900] loss: 0.250


 90%|█████████ | 9/10 [24:40<02:44, 164.85s/it]


[10,   300] loss: 0.251
[10,   600] loss: 0.250
[10,   900] loss: 0.250


100%|██████████| 10/10 [27:28<00:00, 164.89s/it]

Training is finished!


В этом случае разница со скоростью сходимости между разными оптимизаторами еще более заметна.

In [14]:
class FeedForward2(nn.Module):
    # Другая архитектура сети
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward2, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, 2*hidden_dim)
        self.dp1 = nn.Dropout(0.3)
        self.fc2 = Perceptron(2*hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = Perceptron(hidden_dim, 1, activation='sigmoid')
        self.double()
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.dp1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        return x

In [16]:
#Adam
net = FeedForward2(6, 30)

optimizer = torch.optim.Adam(net.parameters())
criterion = RMSELoss()


for epoch in tqdm(range(10)):  
    running_loss = 0.0
    print()
    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels.reshape(1024, -1))
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 299:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('\nTraining is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]


[1,   300] loss: 0.314
[1,   600] loss: 0.252
[1,   900] loss: 0.252


 10%|█         | 1/10 [02:55<26:20, 175.66s/it]


[2,   300] loss: 0.251
[2,   600] loss: 0.251
[2,   900] loss: 0.250


 20%|██        | 2/10 [05:43<22:47, 170.94s/it]

...


[9,   300] loss: 0.248
[9,   600] loss: 0.248
[9,   900] loss: 0.249


 90%|█████████ | 9/10 [25:18<02:47, 167.42s/it]


[10,   300] loss: 0.249
[10,   600] loss: 0.247
[10,   900] loss: 0.248


100%|██████████| 10/10 [28:07<00:00, 168.72s/it]


Training is finished!


Заметной разницы в результатах не наблюдается.